In [ ]:
import pandas as pd
import numpy as np
import anndata
import scanpy as sc
import scipy

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt

sc.set_figure_params(dpi=300, dpi_save=300, frameon=True, vector_friendly=True, fontsize=6, 
figsize=None, color_map=None, format='pdf', facecolor=None, transparent=False, ipython_format='png2x')

### Read data from our model, scSpace and CeLEry

In [ ]:
our_spatial = sc.read_h5ad("ST_recovery_cortexMB.h5ad")
our_seq = sc.read_h5ad("SC_recovery_cortexMB.h5ad")

# our_spatial = sc.read_h5ad("ST_recovery_ME.h5ad")
# our_seq = sc.read_h5ad("SC_recovery_ME.h5ad")

In [ ]:
scSpace_STpos = np.load("../scSpace/scSpace-master/scSpace_infer_STpos.npy")
scSpace_SCpos = np.load("../scSpace/scSpace-master/scSpace_infer_SCpos.npy")

# scSpace_STpos = np.load("../scSpace/scSpace-master/scSpace_inferME_STpos.npy")
# scSpace_SCpos = np.load("../scSpace/scSpace-master/scSpace_inferME_SCpos.npy")

In [ ]:
CeLEry_MB_SC = sc.read_h5ad("../CeLEry_2D/CeLEry-main/CeLEry_package/CeLEry_MB_SC.h5ad")
CeLEry_MB_ST = sc.read_h5ad("../CeLEry_2D/CeLEry-main/CeLEry_package/CeLEry_MB_ST.h5ad")

# CeLEry_ME_SC = sc.read_h5ad("../CeLEry_2D/CeLEry-main/CeLEry_package/CeLEry_ME_SC.h5ad")
# CeLEry_ME_ST = sc.read_h5ad("../CeLEry_2D/CeLEry-main/CeLEry_package/CeLEry_ME_ST.h5ad")

In [ ]:
CeLEry_MB_SC.obsm["scale_infer_spatial"]

In [ ]:
CeLEry_ME_SC

In [ ]:
CeLEry_ME_ST

In [ ]:
scSpace_STpos

In [ ]:
#sc.pl.spatial(scSpace_seq, color="CTX.subclass_id.1", spot_size=0.05, title="Mouse brain spatial celltype") 
#sc.pl.spatial(scSpace_seq, color="celltype_mapped_refined", spot_size=0.05, title="Mouse brain spatial celltype") 

In [ ]:
#sc.pl.spatial(our_spatial, color="CTX.subclass_id.1", spot_size=0.05, title="Mouse brain spatial celltype")
sc.pl.spatial(our_spatial, color="celltype_mapped_refined", spot_size=0.05, title="Mouse brain spatial celltype")

In [ ]:
our_spatial

In [ ]:
our_seq

### For every given cell, calculate the KNN spots that belong to the same cell-type as the cell

In [ ]:
#here we only select the shared common celltype for both scRNA-seq and ST
sc_change = {'CA3-do': "CA3", 'CA3-ve': "CA3"}
our_seq.obs["celltype"] = our_seq.obs["celltype"].replace(sc_change)
common_celltype = set(our_spatial.obs["CTX.subclass_id.1"]).intersection(set(our_seq.obs["celltype"]))

# st_change = {'Presomitic mesoderm': "Somitic mesoderm", 'Splanchnic mesoderm': "Pharyngeal mesoderm", 'Definitive endoderm': 'Def. endoderm',
#             'Gut tube': "Gut", 'Mixed mesenchymal mesoderm': "Mesenchyme"}

# our_spatial.obs["celltype_mapped_refined"] = our_spatial.obs["celltype_mapped_refined"].replace(st_change)

# sc_change = {'Blood progenitors 1': "Blood progenitors", 'Blood progenitors 2': "Blood progenitors", 
#              'Erythroid1': 'Erythroid', 'Erythroid2':'Erythroid', 'Erythroid3': 'Erythroid' }
# our_seq.obs["celltype"] = our_seq.obs["celltype"].replace(sc_change)

# common_celltype = set(our_spatial.obs["celltype_mapped_refined"]).intersection(set(our_seq.obs["celltype"]))

def get_purity_inST(pos_sc, pos_st, sc_celltype, st_celltype, knn_list=[5, 10, 20, 50]):

    obsm_spatial = np.concatenate([pos_sc, pos_st])
    adata_pos = anndata.AnnData(obsm_spatial)

    adata_pos.obs["celltype"] = np.concatenate([sc_celltype, st_celltype])
    adata_pos.obs['celltype'] = pd.Categorical(adata_pos.obs['celltype'])
    adata_pos.obs["modal"] = (["seq"] * pos_sc.shape[0]) + (["spatial"] * pos_st.shape[0])

    # Convert discrete labels to numerical labels
    label_encoder = LabelEncoder()
    #y = label_encoder.fit_transform(adata_concat_envi.obs['celltype'])
    y = label_encoder.fit_transform(adata_pos.obs['celltype'])

    adata_pos.obs["celltype_label"] = y

    adata_pos_st = adata_pos[adata_pos.obs['modal']=='spatial',:]
    adata_pos_sc = adata_pos[adata_pos.obs['modal']=='seq',:]

    for k in knn_list:

        # Initialize the KNeighborsClassifier
        knn = KNeighborsClassifier(n_neighbors=k) # default: euclidian

        # Fit the model (using the same data here for simplicity)
        knn.fit(adata_pos_st.X, adata_pos_st.obs['celltype_label'])

        # Find the k-neighbors for every single cell in scRNA-seq
        distances, indices = knn.kneighbors(adata_pos_sc.X)

        # Calculate the proportion of neighbors in ST belonging to the same celltype for each cell
        proportions = []
        celltype_label_ST = adata_pos_st.obs["celltype_label"]
        celltype_label_SC = adata_pos_sc.obs["celltype_label"]

        for i in range(len(adata_pos_sc.X)):
            # Get the labels of the k-neighbors
            neighbor_labels = celltype_label_ST.iloc[indices[i]]
            # Calculate the proportion of neighbors with the same class as the sample
            proportion = np.sum(neighbor_labels == celltype_label_SC.iloc[i]) / k
            proportions.append(proportion)

        label = "KNN_{}_fraction_inST".format(k)
        adata_pos_sc.obs[label] = proportions

    return adata_pos_sc, adata_pos_st

In [ ]:
np.unique(our_seq.obs['celltype'])

In [ ]:
adata_pos_sc, adata_pos_st = get_purity_inST(our_seq.obsm["spatial"], our_spatial.obsm["raw_spatial"],our_seq.obs["celltype"], our_spatial.obs["CTX.subclass_id.1"])
#adata_pos_sc, adata_pos_st = get_purity_inST(scSpace_SCpos, our_spatial.obsm["raw_spatial"],our_seq.obs["celltype"], our_spatial.obs["CTX.subclass_id.1"])

#adata_pos_sc, adata_pos_st = get_purity_inST(our_seq.obsm["spatial"], our_spatial.obsm["raw_spatial"],our_seq.obs["celltype"], our_spatial.obs["celltype_mapped_refined"])
#adata_pos_sc, adata_pos_st = get_purity_inST(scSpace_SCpos, our_spatial.obsm["raw_spatial"],our_seq.obs["celltype"], our_spatial.obs["celltype_mapped_refined"])
#adata_pos_sc, adata_pos_st = get_purity_inST(CeLEry_ME_SC.obsm["scale_infer_spatial"], our_spatial.obsm["raw_spatial"],our_seq.obs["celltype"], our_spatial.obs["celltype_mapped_refined"])

method = "our"

In [ ]:
_palette = {'CA1-ProS': '#1CE6FF',
 'CA3': '#FF34FF',
 'CA3-do': '#FF4A46',
 'CA3-ve': '#008941',
 'DG': '#006FA6',
 'L2 IT ENTl': '#A30059',
 'L2/3 IT CTX': '#FFDBE5',
 'L4/5 IT CTX': '#7A4900',
 'L5 IT CTX': '#0000A6',
 'L6 CT CTX': '#63FFAC',
 'L6b ENT': '#B79762'}

# _palette = {'Allantois': '#1CE6FF',
# 'Anterior somitic tissues': '#FF34FF',
# 'Blood progenitors': '#FF4A46',
# 'Cardiomyocytes': '#008941',
# 'Caudal Mesoderm': '#006FA6',
# 'Cranial mesoderm': '#A30059',
# 'Def. endoderm': '#FFDBE5',
# 'Dermomyotome': '#7A4900',
# 'Endothelium': '#0000A6',
# 'Erythroid': '#63FFAC',
# 'ExE endoderm': '#B79762',
# 'ExE mesoderm': '#004D43',
# 'Forebrain/Midbrain/Hindbrain': '#8FB0FF',
# 'Gut': '#997D87',
# 'Haematoendothelial progenitors': '#5A0007',
# 'Intermediate mesoderm': '#809693',
# 'Lateral plate mesoderm': '#6A3A4C',
# 'Low quality': '#1B4400',
# 'Mesenchyme': '#4FC601',
# 'NMP': '#3B5DFF',
# 'Neural crest': '#4A3B53',
# 'Notochord': '#FF2F80',
# 'PGC': '#61615A',
# 'Paraxial mesoderm': '#BA0900',
# 'Pharyngeal mesoderm': '#6B7900',
# 'Rostral neurectoderm': '#00C2A0',
# 'Sclerotome': '#FFAA92',
# 'Somitic mesoderm': '#FF90C9',
# 'Spinal cord': '#B903AA',
# 'Surface ectoderm': '#D16100',
# 'Visceral endoderm': '#DDEFFF'}

In [ ]:
from matplotlib import cm, colors
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager # to solve: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
import json

centimeter = 1/2.54  # centimeter in inches

# https://www.geeksforgeeks.org/react-js-blueprint-colors-qualitative-color-schemes/
react_cols_10 = ['#147EB3','#29A634','#D1980B','#D33D17','#9D3F9D','#00A396','#DB2C6F','#8EB125','#946638','#7961DB']

# http://tsitsul.in/blog/coloropt/
norm_7 = ['#4053d3','#ddb310','#b51d14','#00beff','#fb49b0','#00b25d','#cacaca']
norm_12 = ['#ebac23','#b80058','#008cf9','#006e00','#00bbad','#d163e6','#b24502',
           '#ff9287','#5954d6','#00c6f8','#878500','#00a76c','#bdbdbd']

def config_rc(dpi=400, font_size=6, lw=1.):
    # matplotlib.rcParams.keys()
    rc={
        'font.size': font_size, 
        'axes.labelsize': font_size, 
        'axes.titlesize': font_size, 
        'xtick.labelsize': font_size, 
        'ytick.labelsize': font_size,
        'figure.dpi':dpi,'axes.linewidth':lw,
        'legend.markerscale': 0.8, 
        'legend.markerscale': 0.8, 
        'legend.loc': 'upper right',
        'legend.borderpad':0.2,
        'legend.columnspacing': 0.5,
        'legend.labelspacing': 0.2,
        'legend.handletextpad': 0.1,
        'legend.borderaxespad': 0.1,
        'legend.handleheight': 1.0,
        'legend.handlelength': 1.0,
    } # 'figure.figsize':(11.7/1.5,8.27/1.5)
    
    sns.set(style='ticks',rc=rc) 
    sns.set_context("paper")

    mpl.rcParams.update(rc)

    mpl.rcParams['pdf.fonttype'] = 42
    mpl.rcParams['ps.fonttype'] = 42

    #mpl.rcParams['font.sans-serif'] = "Arial"
    mpl.rcParams['font.family'] = "sans-serif"
    mpl.rcParams['axes.unicode_minus']=False # negative minus sign

In [ ]:
df = adata_pos_sc.obs
subset_df = df[df['celltype'].isin(common_celltype)]
subset_df = subset_df.reset_index(drop=True)
categories_to_remove = set(adata_pos_sc.obs["celltype"]) - common_celltype
subset_df['celltype'] = subset_df['celltype'].cat.remove_categories(list(categories_to_remove))
ground_truth = (adata_pos_st.obs['celltype'].value_counts()/len(adata_pos_st)).to_dict()

config_rc(dpi=300, font_size=6, lw=1.)

for k in [5, 10, 20, 50]:

    fig, ax = plt.subplots(figsize=(8*centimeter, 3*centimeter))

    sns.boxplot(x='celltype', y='KNN_{}_fraction_inST'.format(k), data=subset_df, ax=ax, palette=_palette, dodge=False, width=0.5, showfliers=False)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')

    ordered_celltype = subset_df['celltype'].cat.categories.values

    for celltype in ordered_celltype:
        ax.scatter([celltype], [ground_truth[celltype]], s=3, marker="^", color="r") #, edgecolors="black"

    #fig.savefig("plots/{}_sc_recovery_boxplt_ME.pdf".format(method), bbox_inches='tight', format='pdf', dpi=300)

In [ ]:
subset_df.groupby(["celltype"]).mean(numeric_only=True)


In [ ]:
df_scspace = subset_df

In [ ]:
subset_df[['KNN_5_fraction_inST', 'KNN_10_fraction_inST', 'KNN_20_fraction_inST', 'KNN_50_fraction_inST']].mean()

In [ ]:
import scipy.stats as stats
for i in ['KNN_5_fraction_inST', 'KNN_10_fraction_inST', 'KNN_20_fraction_inST', 'KNN_50_fraction_inST']:
    ## perform U test between our RMSE and other methods:
    print("running {}",i)
    #print(stats.mannwhitneyu(df_our[i], df_celery[i]))
    print(stats.mannwhitneyu(df_our[i], df_scspace[i]))

In [ ]:
our_spatial

In [ ]:
# our_spatial.obsm['spatial'] = scSpace_STpos
# our_seq.obsm['spatial'] = scSpace_SCpos

#method = "CeLEry"
our_spatial.obsm['spatial'] = CeLEry_ME_ST.obsm["scale_infer_spatial"]
our_seq.obsm['spatial'] = CeLEry_ME_SC.obsm["spatial"]

In [ ]:
fig = plt.figure(figsize=(4*centimeter, 4*centimeter), dpi=300)
ax = plt.subplot(aspect = 'equal')
#sc.pl.spatial(our_spatial, color="CTX.subclass_id.1", palette=_palette, spot_size=0.05, legend_loc=None, ax=ax, title="Infered psudo-space for ST")
#sc.pl.spatial(our_spatial, color="CTX.subclass_id.1", palette=_palette, spot_size=0.01, legend_loc=None, ax=ax, title="Infered psudo-space for ST")

sc.pl.spatial(our_spatial, color="celltype_mapped_refined", palette=_palette, spot_size=0.05, legend_loc=None, ax=ax, title="Infered psudo-space for ST")


fig.savefig("plots/{}_ME_spatialMap_STinfer.pdf".format(method), bbox_inches='tight', format='pdf', dpi=300)

In [ ]:
fig = plt.figure(figsize=(4*centimeter, 4*centimeter), dpi=300)
ax = plt.subplot(aspect = 'equal')
sc.pl.spatial(our_seq, color="celltype",palette=_palette, spot_size=0.05, legend_loc=None, title="Infered psudo-space for SC", ax=ax)  #

fig.savefig("plots/{}_ME_spatialMap_SCinfer.pdf".format(method), bbox_inches='tight', format='pdf', dpi=300)